# Część 2 - Podstawy Spark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("talion").getOrCreate()

24/06/03 17:12:00 WARN Utils: Your hostname, AndrzejSzlaski resolves to a loopback address: 127.0.1.1; using 172.21.92.194 instead (on interface eth0)
24/06/03 17:12:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/03 17:12:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print("Spark version:", spark.version)

Spark version: 3.5.1


In [4]:
data = [1, 2, 3, 4, 5]

rdd = spark.sparkContext.parallelize(data)

print("RDD collect:", rdd.collect())

RDD collect: [1, 2, 3, 4, 5]


In [5]:
df = spark.createDataFrame([("Andorra", 98), ("United Arab Emirates", 1683)], ["country_name", "total_confirmed"])

df.show()

+--------------------+---------------+
|        country_name|total_confirmed|
+--------------------+---------------+
|             Andorra|             98|
|United Arab Emirates|           1683|
+--------------------+---------------+



## 2.1 Wczytaj dane ze swojego pliku CSV dotyczącego danych COVID-19 (pamiętaj o nagłówku dla kolumn).

In [6]:
path = 'spark/covid_ms_gold.csv'
df = spark.read.option("delimiter", ",").option("header", True).csv(path)
df = df.select([c for c in df.columns if c in ["location_key", "country_name", "date", "cumulative_confirmed", "new_confirmed", "cumulative_tested", "new_tested", "cumulative_deceased", "new_deceased", "2022 Population", "cumulative_persons_vaccinated", "new_persons_vaccinated", "Area (km²)"]])
df.show(1)

+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+
|location_key|country_name|      date|cumulative_confirmed|new_confirmed|cumulative_tested|new_tested|cumulative_deceased|new_deceased|cumulative_persons_vaccinated|new_persons_vaccinated|2022 Population|Area (km²)|
+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+
|          AD|     Andorra|2020-01-01|                   0|            0|             NULL|      NULL|                  0|           0|                         NULL|                  NULL|        79824.0|     468.0|
+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-

## 2.2 Utwórz DataFrame. Pamiętaj o utworzeniu własnego schematu z wybranymi kolumnami.

In [7]:
df.printSchema()

root
 |-- location_key: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- cumulative_confirmed: string (nullable = true)
 |-- new_confirmed: string (nullable = true)
 |-- cumulative_tested: string (nullable = true)
 |-- new_tested: string (nullable = true)
 |-- cumulative_deceased: string (nullable = true)
 |-- new_deceased: string (nullable = true)
 |-- cumulative_persons_vaccinated: string (nullable = true)
 |-- new_persons_vaccinated: string (nullable = true)
 |-- 2022 Population: string (nullable = true)
 |-- Area (km²): string (nullable = true)



In [8]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from pyspark.sql.functions import col

In [9]:
df = df.withColumn("location_key", col("location_key").cast(StringType()))
df = df.withColumn("country_name", col("country_name").cast(StringType()))
df = df.withColumn("date", col("date").cast(DateType()))
df = df.withColumn("cumulative_confirmed", col("cumulative_confirmed").cast(IntegerType()))
df = df.withColumn("new_confirmed", col("new_tested").cast(IntegerType()))
df = df.withColumn("cumulative_tested", col("cumulative_tested").cast(IntegerType()))
df = df.withColumn("new_tested", col("new_tested").cast(IntegerType()))
df = df.withColumn("cumulative_deceased", col("cumulative_deceased").cast(IntegerType()))
df = df.withColumn("new_deceased", col("new_deceased").cast(IntegerType()))
df = df.withColumn("cumulative_persons_vaccinated", col("cumulative_persons_vaccinated").cast(IntegerType()))
df = df.withColumn("new_persons_vaccinated", col("new_persons_vaccinated").cast(IntegerType()))
df = df.withColumn("2022 Population", col("2022 Population").cast(IntegerType()))
df = df.withColumn("Area (km²)", col("Area (km²)").cast(DoubleType()))

In [10]:
df.printSchema()

root
 |-- location_key: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- cumulative_confirmed: integer (nullable = true)
 |-- new_confirmed: integer (nullable = true)
 |-- cumulative_tested: integer (nullable = true)
 |-- new_tested: integer (nullable = true)
 |-- cumulative_deceased: integer (nullable = true)
 |-- new_deceased: integer (nullable = true)
 |-- cumulative_persons_vaccinated: integer (nullable = true)
 |-- new_persons_vaccinated: integer (nullable = true)
 |-- 2022 Population: integer (nullable = true)
 |-- Area (km²): double (nullable = true)



## 2.3 Wyświetl co najmniej 5 kolumn i wyjaśnij ich znaczenie dla analizy przypadków COVID-19.

In [11]:
df.select('location_key', 'country_name', 'date', 'new_confirmed', 'new_tested', 'new_deceased').show(10)

+------------+------------+----------+-------------+----------+------------+
|location_key|country_name|      date|new_confirmed|new_tested|new_deceased|
+------------+------------+----------+-------------+----------+------------+
|          AD|     Andorra|2020-01-01|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-02|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-03|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-04|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-05|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-06|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-07|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-08|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-09|         NULL|      NULL|           0|
|          AD|     Andorra|2020-01-10|         NULL|      NULL|           0|

## 2.4 Dodaj kolumnę, która będzie zbierała dane, które uznasz za stosowne i które są istotne z punktu widzenia analizy COVID-19 (np. wyliczenie gęstości zaludnienia na podstawie liczby ludności i powierzchni kraju).

In [12]:
df = df.withColumn("population_density", col("2022 Population") / col("Area (km²)"))

In [13]:
df.select('country_name', 'population_density').distinct().orderBy('country_name').show()

+-------------------+------------------+
|       country_name|population_density|
+-------------------+------------------+
|        Afghanistan| 63.05869248577956|
|            Albania| 98.87021705857799|
|            Algeria|18.853109972914773|
|     American Samoa|222.47738693467338|
|            Andorra|170.56410256410257|
|             Angola|28.546552498596295|
|           Anguilla|174.25274725274724|
|         Antarctica|              NULL|
|Antigua and Barbuda|212.13348416289594|
|          Argentina|16.368262839879154|
|            Armenia| 93.48313888982281|
|              Aruba| 591.3611111111111|
|          Australia|3.4031891996176817|
|            Austria|106.58770015857687|
|         Azerbaijan|119.60824480369514|
|            Bahamas| 29.40428889048268|
|            Bahrain|1924.4875816993465|
|         Bangladesh|1160.0350477739378|
|           Barbados| 654.9651162790698|
|            Belarus|45.929450867052026|
+-------------------+------------------+
only showing top

## 2.5 Połącz wybrane kolumny ze sobą w sposób niosący nowe informacje za pomocą funkcji concat . 

Stworzona została kolumna id która jednoznacznie identyfikuje dany dzień dla danego kraju

In [14]:
df.show()

+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+------------------+
|location_key|country_name|      date|cumulative_confirmed|new_confirmed|cumulative_tested|new_tested|cumulative_deceased|new_deceased|cumulative_persons_vaccinated|new_persons_vaccinated|2022 Population|Area (km²)|population_density|
+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+------------------+
|          AD|     Andorra|2020-01-01|                   0|         NULL|             NULL|      NULL|                  0|           0|                         NULL|                  NULL|          79824|     468.0|170.56410256410257|
|          AD|     Andorra|2020-01-02|                   0| 

In [15]:
from pyspark.sql.functions import concat, lit

df = df.withColumn('id', concat(col('location_key'), lit('_'), col('date')))
df.show()

+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+------------------+-------------+
|location_key|country_name|      date|cumulative_confirmed|new_confirmed|cumulative_tested|new_tested|cumulative_deceased|new_deceased|cumulative_persons_vaccinated|new_persons_vaccinated|2022 Population|Area (km²)|population_density|           id|
+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+------------------+-------------+
|          AD|     Andorra|2020-01-01|                   0|         NULL|             NULL|      NULL|                  0|           0|                         NULL|                  NULL|          79824|     468.0|170.56410256410257|AD_2020-01-01|
|   

## 2.6 Wyfiltruj co najmniej 10 wybranych przez siebie, istotnych informacji (np. kraj z liczbą łóżek większą od 10000 czy dzień, w którym było najmniej zachorowań w danym kraju).

Kraje które korygowały statystyke śmierci z użyciem ujemnych danych

In [16]:
df.filter("new_deceased < 0").select('country_name').distinct().show()

+------------+
|country_name|
+------------+
|     Uruguay|
|     Mayotte|
|    Botswana|
|     Romania|
|      Belize|
|     Bahamas|
|     Réunion|
|Sierra Leone|
|      France|
|       Italy|
|    Honduras|
|      Taiwan|
|       China|
|  Bangladesh|
|     Jamaica|
|     Comoros|
|   Sri Lanka|
|   Guatemala|
+------------+



Kraje z niskim zagęszczeniem ludności

In [17]:
df.filter("population_density < 10").select('country_name').distinct().show()

+--------------------+
|        country_name|
+--------------------+
|             Iceland|
|            Botswana|
|          Mauritania|
|Central African R...|
|          Kazakhstan|
|            Mongolia|
|               Libya|
|               Gabon|
|             Namibia|
|      Western Sahara|
|           Australia|
|              Canada|
|            Suriname|
|              Russia|
|    Falkland Islands|
|           Greenland|
|              Guyana|
|       French Guiana|
|                Niue|
+--------------------+



Kraje z wysokim zagęszczeniem ludności

In [18]:
df.filter("population_density > 10000").select('country_name').distinct().show()

+------------+
|country_name|
+------------+
|      Monaco|
|       Macau|
+------------+



Dane o pandemii dla Polski

In [19]:
df.filter("location_key == 'PL'").show()

+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+------------------+-------------+
|location_key|country_name|      date|cumulative_confirmed|new_confirmed|cumulative_tested|new_tested|cumulative_deceased|new_deceased|cumulative_persons_vaccinated|new_persons_vaccinated|2022 Population|Area (km²)|population_density|           id|
+------------+------------+----------+--------------------+-------------+-----------------+----------+-------------------+------------+-----------------------------+----------------------+---------------+----------+------------------+-------------+
|          PL|      Poland|2020-01-01|                   0|         NULL|             NULL|      NULL|                  0|           0|                         NULL|                  NULL|       39857145|  312679.0|127.46984927033795|PL_2020-01-01|
|   

Kraje w którch nowe zarażenia jakiegoś dnia przekroczyły milion

In [20]:
df.filter("new_confirmed > 1000000").select('country_name').distinct().show()

+--------------------+
|        country_name|
+--------------------+
|            Mongolia|
|               Japan|
|              France|
|               Italy|
|           Australia|
|             Austria|
|               India|
|         South Korea|
|            Pakistan|
|      United Kingdom|
|              Russia|
|United States of ...|
+--------------------+



Kraje w którch zarażenia skumulowane były mniejsze od nowych co oznacza błąd w prowadzeniu danych

In [21]:
df.filter("new_confirmed > cumulative_confirmed").select('country_name').distinct().show()

+--------------------+
|        country_name|
+--------------------+
|             Belgium|
|             Belarus|
|             Bolivia|
|             Morocco|
|             Iceland|
|             Uruguay|
|              Mexico|
|            Zimbabwe|
|United Arab Emirates|
|            Botswana|
|             Bahrain|
|            Paraguay|
|              Malawi|
|           Argentina|
|             Myanmar|
|          Mauritania|
|                Laos|
|        Saudi Arabia|
|            Ethiopia|
|              Brazil|
+--------------------+
only showing top 20 rows



Kraje w którch zbierano dane o testach na Covid

In [22]:
df.filter(col("new_tested").isNotNull()).select('country_name').distinct().show()

+--------------------+
|        country_name|
+--------------------+
|   Equatorial Guinea|
|             Belgium|
|             Belarus|
|             Bolivia|
|             Morocco|
|             Iceland|
|             Uruguay|
|              Mexico|
|            Zimbabwe|
|United Arab Emirates|
|            Botswana|
|        Burkina Faso|
|             Bahrain|
|            Paraguay|
|              Malawi|
|           Argentina|
|             Myanmar|
|      Cayman Islands|
|          Mauritania|
|            Barbados|
+--------------------+
only showing top 20 rows



Kraje w którch zarażenia wpisano już pierwszego dnia z którego zbierane są dane

In [23]:
df.filter("date == '2020-01-01'").filter(col('new_confirmed').isNotNull()).select('country_name', 'new_confirmed').orderBy('new_confirmed', ascending=False).show()

+------------+-------------+
|country_name|new_confirmed|
+------------+-------------+
|      Brazil|         1154|
|      Mexico|           72|
|   Argentina|            4|
+------------+-------------+



Kraj z najwyższą liczbą skumulowanych zarażeń

In [24]:
df.filter(col('cumulative_persons_vaccinated').isNotNull()).select('country_name', 'cumulative_persons_vaccinated').orderBy('cumulative_persons_vaccinated', ascending=False).show(1)

+------------+-----------------------------+
|country_name|cumulative_persons_vaccinated|
+------------+-----------------------------+
|       China|                   1302773000|
+------------+-----------------------------+
only showing top 1 row



Kraj z najwyższą liczbą nowych zarażeń

In [25]:
df.filter(col('new_persons_vaccinated').isNotNull()).select('country_name', 'new_persons_vaccinated').orderBy('new_persons_vaccinated', ascending=False).show(1)

+------------+----------------------+
|country_name|new_persons_vaccinated|
+------------+----------------------+
|       China|             450500000|
+------------+----------------------+
only showing top 1 row



## 2.7 Wykonaj 5 agregacji na swoim DF (np. min, max, agg, count, mean) uprzednio pogrupowawszy kolumny (groupBy).

In [26]:
from pyspark.sql.functions import min, max, count, mean, sum

In [27]:
agg_df = df.groupBy("country_name").agg(
    min("new_confirmed").alias("min_new_confirmed"),
    max("new_confirmed").alias("max_new_confirmed"),
    count("new_confirmed").alias("count_new_confirmed"),
    mean("new_confirmed").alias("mean_new_confirmed"),
    sum("new_confirmed").alias("sum_new_confirmed")
)
agg_df.orderBy('country_name').show()

+-------------------+-----------------+-----------------+-------------------+------------------+-----------------+
|       country_name|min_new_confirmed|max_new_confirmed|count_new_confirmed|mean_new_confirmed|sum_new_confirmed|
+-------------------+-----------------+-----------------+-------------------+------------------+-----------------+
|        Afghanistan|             NULL|             NULL|                  0|              NULL|             NULL|
|            Albania|                1|             8746|                720|2241.4861111111113|          1613870|
|            Algeria|             NULL|             NULL|                  0|              NULL|             NULL|
|     American Samoa|             NULL|             NULL|                  0|              NULL|             NULL|
|            Andorra|             NULL|             NULL|                  0|              NULL|             NULL|
|             Angola|             1459|            13038|                  5|   

# 4 Misja dodatkowa - Mapowanie

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType

df_to_normalize = df.select('location_key', 'country_name', 'date', 'new_confirmed', 'new_tested', 'new_deceased', '2022 Population')

schema = StructType([
    StructField("location_key", StringType(), True),
    StructField("country_name", StringType(), True),
    StructField("date", DateType(), True),
    StructField("population", LongType(), True),
    StructField("cases", DoubleType(), True),
    StructField("deaths", DoubleType(), True),
    StructField("tests", DoubleType(), True)
])

def normalize(row):
    location_key = row['location_key']
    country_name = row['country_name']
    date = row['date']
    population = row['2022 Population']
    
    normalized_cases = None
    normalized_deaths = None
    normalized_tests = None

    if population and population != 0:
        if row['new_confirmed'] is not None:
            normalized_cases = (row['new_confirmed'] / population) * 1000
        if row['new_deceased'] is not None:
            normalized_deaths = (row['new_deceased'] / population) * 1000
        if row['new_tested'] is not None:
            normalized_tests = (row['new_tested'] / population) * 1000

    return (location_key, country_name, date, population, normalized_cases, normalized_deaths, normalized_tests)

normalized_rdd = df_to_normalize.rdd.map(normalize)
normalized_df = spark.createDataFrame(normalized_rdd, schema)

normalized_df.show()

+------------+------------+----------+----------+-----+------+-----+
|location_key|country_name|      date|population|cases|deaths|tests|
+------------+------------+----------+----------+-----+------+-----+
|          AD|     Andorra|2020-01-01|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-02|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-03|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-04|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-05|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-06|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-07|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-08|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-09|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-10|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01-11|     79824| NULL|   0.0| NULL|
|          AD|     Andorra|2020-01

# zapis do csv

In [36]:
output_path1 = "/spark/csv_df"
output_path2 = "/spark/csv_norm_df"
df.write.csv(output_path1, header=True, mode="overwrite")
normalized_df.write.csv(output_path2, header=True, mode="overwrite")